# Bloqs

Bloqs lets you represent high-level quantum programs and subroutines as a hierarchical
collection of Python objects. The main interface is the `Bloq` abstract base class.

In [ ]:
import abc
from typing import *
import attrs
from dataclasses import dataclass

In [ ]:
from cirq_qubitization.quantum_graph.fancy_registers import FancyRegister, Side

In [ ]:
import attrs
from cirq_qubitization.quantum_graph.bloq import Bloq
from cirq_qubitization.quantum_graph.fancy_registers import FancyRegisters

In [ ]:
from cirq_qubitization.jupyter_tools import show_bloq

In [ ]:
class Swap(Bloq):
    @property
    def registers(self):
        return FancyRegisters.build(x=1, y=1)
    
swap = Swap()
show_bloq(swap)

In [ ]:
@dataclass(frozen=True)
class ModExp(Bloq):
    n: int
    
    @property
    def registers(self):
        return FancyRegisters([
            FancyRegister('exp', bitsize=2*self.n),
            FancyRegister('result', bitsize=self.n, side=Side.RIGHT)
        ])
    
modexp = ModExp(n=1024)
show_bloq(modexp)

In [ ]:
class SwapTwoBits(Bloq):
    @property
    def registers(self):
        return FancyRegisters([
            FancyRegister('x', 1),
            FancyRegister('y', 1),
        ])
    
    def short_name(self):
        return '⇋'

In [ ]:
swap = SwapTwoBits()
show_bloq(swap)

In [ ]:
from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloqBuilder
from cirq_qubitization.bloq_algos.basic_gates import CNOT

In [ ]:
class SwapTwoBits(Bloq):
    @property
    def registers(self):
        return FancyRegisters.build(x=1, y=1)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder',
        *, x, y
    ):
        x, y = bb.add(CNOT(), ctrl=x, target=y)
        y, x = bb.add(CNOT(), ctrl=y, target=x)
        x, y = bb.add(CNOT(), ctrl=x, target=y)
        return {'x': x, 'y': y}

In [ ]:
swap = SwapTwoBits()
show_bloq(swap.decompose_bloq())

In [ ]:
cbloq = swap.as_composite_bloq()
from cirq_qubitization.quantum_graph.musical_score import get_musical_score_data, draw_musical_score, dump_musical_score
msd = get_musical_score_data(cbloq)
draw_musical_score(msd)
dump_musical_score(msd, 'tutorial-swap')

In [ ]:
cbloq = swap.decompose_bloq()
msd = get_musical_score_data(cbloq)
draw_musical_score(msd)
dump_musical_score(msd, 'tutorial-swap-cbloq')

## Larger registers

Our two bloqs have still been operating at the level of individual bits. We now consider
a general swap between two `n`-sized registers.

In [ ]:
@attrs.frozen
class Swap(Bloq):
    n: int

    @property
    def registers(self):
        return FancyRegisters([
            FancyRegister('x', bitsize=self.n),
            FancyRegister('y', bitsize=self.n),
        ])

In [ ]:
show_bloq(Swap(n=1000))

In [ ]:
from cirq_qubitization.quantum_graph.composite_bloq import SoquetT

In [ ]:
@attrs.frozen
class Swap(Bloq):
    n: int

    @property
    def registers(self):
        return FancyRegisters.build(x=self.n, y=self.n)

    def build_composite_bloq(
            self, bb: 'CompositeBloqBuilder', *, x: SoquetT, y: SoquetT
    ) -> Dict[str, SoquetT]:
        xs = bb.split(x)
        ys = bb.split(y)

        for i in range(self.n):
            xs[i], ys[i] = bb.add(SwapTwoBits(), x=xs[i], y=ys[i])
        return {
            'x': bb.join(xs),
            'y': bb.join(ys),
        }

In [ ]:
cbloq = Swap(n=5).decompose_bloq()
show_bloq(cbloq)

In [ ]:
cbloq = Swap(n=5).as_composite_bloq()
msd = get_musical_score_data(cbloq)
draw_musical_score(msd)
dump_musical_score(msd, 'tutorial-multiswap')

## Bloq protocols

Bloqs support a growing list of protocols that let you annotate a given `Bloq` with more
definitions or known information. In the following table we summarize the available protocols. Please note that the method you override as a bloq writer is often different than the method you call as a bloq user.

<table>
<thead>
<tr>
<th>What</th>
<th>Call this</th>
<th>Override this</th>
<th>Notebook</th>
</tr>
</thead>
<tbody>
    
<tr><td style='text-align: left;'>Bloq decomposition</td>
<td><code>decompose_bloq()</code></td>
<td><code>build_composite_bloq(...)</code></td>
<td></td>
</tr>

<tr><td style='text-align: left'>Numerical simulation via<br/>quimb tensor networks</td>
<td><code>tensor_contract()</code></td>
<td><code>add_my_tensors(...)</code></td>
<td></td>
</tr>
    
<tr><td style='text-align: left'>Classical simulation</td>
<td><code>call_classically(**vals)</code></td>
<td><code>on_classical_vals(...)</code></td>
<td>classical_sim.ipynb</td>
</tr>
    
<tr><td style='text-align: left'>Resource counting</td>
<td colspan='2' style='text-align: center'><code>t_complexity()</code> for both</td>
<td></td>
</tr>
    
<tr><td style='text-align: left'>Conversion to Cirq</td>
<td><code>to_cirq_circuit(**quregs)</code></td>
<td><code>on_registers(...)</code></td>
<td>cirq_gate.ipynb</td>
</tr>
    
</tbody></table>

In [ ]:
cbloq = Swap(n=5).decompose_bloq()
msd = get_musical_score_data(cbloq)
draw_musical_score(msd)
dump_musical_score(msd, 'tutorial-multiswap-cbloq')

In [ ]:
cbloq = Swap(n=5).decompose_bloq().flatten_once(lambda b: isinstance(b.bloq, SwapTwoBits))
msd = get_musical_score_data(cbloq)
draw_musical_score(msd)
dump_musical_score(msd, 'tutorial-multiswap-cbloq-flat')

## selectively decompose?

In [ ]:
cbloq = Swap(n=5).as_composite_bloq()
draw_musical_score(get_musical_score_data(cbloq))

cbloq = cbloq.flatten_once(lambda b: b.i == 0)
draw_musical_score(get_musical_score_data(cbloq))

cbloq = cbloq.flatten_once(lambda b: b.i == 3)
draw_musical_score(get_musical_score_data(cbloq))

cbloq = cbloq.flatten_once(lambda b: b.i == 5)
draw_musical_score(get_musical_score_data(cbloq))

# Tensors

In [ ]:
import numpy as np

In [ ]:
swap_tensor = np.eye(2**2)   \
    .reshape((2,) * 2 * 2)   \
    .transpose([0, 3, 1, 2])

swap_tensor.reshape(4,4)

In [ ]:
cswap_tensor = np.eye(2**3).reshape((2,) * 3 * 2)
cswap_tensor[1, :, :, 1, ::] = swap_tensor
cswap_tensor.reshape(8,8)

In [ ]:
import quimb.tensor as qtn

In [ ]:
@attrs.frozen
class TwoBitCSwap(Bloq):
    @property
    def registers(self) -> FancyRegisters:
        return FancyRegisters.build(ctrl=1, x=1, y=1)

    def add_my_tensors(
        self, tn: 'qtn.TensorNetwork', tag: Any, *,
        incoming: Dict[str, 'SoquetT'], outgoing: Dict[str, 'SoquetT']
    ):
        out_inds = [outgoing['ctrl'], outgoing['x'], outgoing['y']]
        in_inds = [incoming['ctrl'], incoming['x'], incoming['y']]
        
        tn.add(qtn.Tensor(data=cswap_tensor, inds=out_inds + in_inds))

In [ ]:
from cirq_qubitization.bloq_algos.basic_gates import ZeroEffect, ZeroState, OneEffect, OneState

In [ ]:
def set_ctrl_two_bit_swap(ctrl_bit):
    states = [ZeroState(), OneState()]
    effs = [ZeroEffect(), OneEffect()]

    bb = CompositeBloqBuilder()
    (q0,) = bb.add(states[ctrl_bit])
    q1 = bb.add_register('q1', 1)
    q2 = bb.add_register('q2', 1)
    q0, q1, q2 = bb.add(TwoBitCSwap(), ctrl=q0, x=q1, y=q2)
    bb.add(effs[ctrl_bit], q=q0)
    return bb.finalize(q1=q1, q2=q2)

cbloq = set_ctrl_two_bit_swap(ctrl_bit=0)
show_bloq(cbloq)

In [ ]:
cbloq.tensor_contract()

In [ ]:
set_ctrl_two_bit_swap(ctrl_bit=1) \
    .tensor_contract()

In [ ]:
@attrs.frozen
class CSwap(Bloq):


    bitsize: int

    @property
    def registers(self) -> FancyRegisters:
        return FancyRegisters.build(ctrl=1, x=self.bitsize, y=self.bitsize)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder', ctrl: 'SoquetT', x: 'SoquetT', y: 'SoquetT'
    ) -> Dict[str, 'SoquetT']:
        xs = bb.split(x)
        ys = bb.split(y)

        for i in range(self.bitsize):
            ctrl, xs[i], ys[i] = bb.add(TwoBitCSwap(), ctrl=ctrl, x=xs[i], y=ys[i])

        return {'ctrl': ctrl, 'x': bb.join(xs), 'y': bb.join(ys)}

In [ ]:
@attrs.frozen
class ModExp(Bloq):
    base: int
    mod: int
    exp_bitsize: int
    x_bitsize: int

    @property
    def registers(self) -> 'FancyRegisters':
        return FancyRegisters([
            FancyRegister('exponent', bitsize=self.exp_bitsize),
            FancyRegister('x', bitsize=self.x_bitsize, side=Side.RIGHT),
        ])

    def on_classical_vals(self, exponent: int):
        return {'exponent': exponent, 
                'x': (self.base**exponent) % self.mod}

In [ ]:
modexp = ModExp(base=8, mod=13*17, exp_bitsize=8, x_bitsize=8)

for exp in range(10):
    exponent, f = modexp.call_classically(exponent=exp)
    print(exponent, f)

In [ ]:
@attrs.frozen
class ModExp(Bloq):
    base: int
    mod: int
    exp_bitsize: int
    x_bitsize: int

    @property
    def registers(self) -> 'FancyRegisters':
        return FancyRegisters(
            [
                FancyRegister('exponent', bitsize=self.exp_bitsize),
                FancyRegister('x', bitsize=self.x_bitsize, side=Side.RIGHT),
            ]
        )

    def short_name(self) -> str:
        return f'{self.base}^e % {self.mod}'

    def on_classical_vals(self, exponent: int):
        return {'exponent': exponent, 
                'x': (self.base**exponent) % self.mod}

    def CtrlModMul(self, k: int):
        return CtrlModMul(k=k, bitsize=self.x_bitsize, mod=self.mod)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder', exponent: 'SoquetT'
    ) -> Dict[str, 'SoquetT']:
        (x,) = bb.add(IntState(val=1, bitsize=self.x_bitsize))
        exponent = bb.split(exponent)

        # https://en.wikipedia.org/wiki/Modular_exponentiation#Right-to-left_binary_method
        base = self.base
        for j in range(self.exp_bitsize - 1, 0 - 1, -1):
            exponent[j], x = bb.add(self.CtrlModMul(k=base), ctrl=exponent[j], x=x)
            base = base * base % self.mod

        return {'exponent': bb.join(exponent), 'x': x}